In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import math

In [2]:
url = 'https://www.futfanatics.com.br/loja/catalogo.php?loja=311840&categoria=56&variacao=tamanho_41&range=&order=2&pg=1'

headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"}

site = requests.get(url, headers=headers)

encoding = site.encoding if 'charset' in site.headers.get('content-type', '').lower() else None
parser = 'html5lib'
soup = BeautifulSoup(site.content, parser, from_encoding=encoding)

totais = soup.find_all('div', class_="search-counter")

total_texto = totais[0].get_text().strip()

index = total_texto.find(' ')

qtd  = total_texto[:index]

ultima_pagina = math.ceil(int(qtd)/40)

print("Quantidade de produtos:",qtd)
print("Quantidade de páginas:",ultima_pagina)

Quantidade de produtos: 1320
Quantidade de páginas: 33


In [3]:
dic_produtos = {'marca':[], 'preco':[], 'desconto':[], 'link':[]}

for i in range(1, ultima_pagina+1):
    print('Página', i, 'de', ultima_pagina)
    url_pag = f'https://www.futfanatics.com.br/loja/catalogo.php?loja=311840&categoria=56&variacao=tamanho_41&range=&order=2&pg={i}'
    site = requests.get(url_pag, headers=headers)
    soup = BeautifulSoup(site.content, parser, from_encoding=encoding)
    produtos = soup.find_all('div', class_=re.compile('product-item'))

    for produto in produtos:
        marca = produto.find('div', class_=re.compile('product-name')).get_text().strip()
        preco = produto.find('div', class_='price').get_text().strip()
        desconto = 0
        descontos = produto.find_all('span', class_=re.compile('discount'))
        if len(descontos) > 0:
            desconto = descontos[0].get_text().strip()
        link = produto.find_all('a', href=True)[0]['href']
        dic_produtos['marca'].append(marca)
        dic_produtos['preco'].append(preco)
        dic_produtos['desconto'].append(desconto)
        dic_produtos['link'].append(link)

df = pd.DataFrame(dic_produtos)

Página 1 de 33
Página 2 de 33
Página 3 de 33
Página 4 de 33
Página 5 de 33
Página 6 de 33
Página 7 de 33
Página 8 de 33
Página 9 de 33
Página 10 de 33
Página 11 de 33
Página 12 de 33
Página 13 de 33
Página 14 de 33
Página 15 de 33
Página 16 de 33
Página 17 de 33
Página 18 de 33
Página 19 de 33
Página 20 de 33
Página 21 de 33
Página 22 de 33
Página 23 de 33
Página 24 de 33
Página 25 de 33
Página 26 de 33
Página 27 de 33
Página 28 de 33
Página 29 de 33
Página 30 de 33
Página 31 de 33
Página 32 de 33
Página 33 de 33


In [4]:
df.head()

,marca,preco,desconto,link
0,Tenis New Balance Fresh Foam X 1080 Unlaced Prata,"R$1.299,90",0,https://www.futfanatics.com.br/tenis-new-balan...
1,Tenis New Balance Fresh Foam X 1080 Unlaced Br...,"R$1.299,90",0,https://www.futfanatics.com.br/tenis-new-balan...
2,Tênis New Balance Fresh Foam X 1080 Unlaced Preto,"R$1.299,90",0,https://www.futfanatics.com.br/tenis-new-balan...
3,Tênis New Balance More v4 Azul,"R$1.249,90",0,https://www.futfanatics.com.br/tenis-new-balan...
4,Tênis New Balance 1080 V12 London Edition Preto,"R$1.249,90",0,https://www.futfanatics.com.br/tenis-new-balan...


In [5]:
tabela = df.copy()

tabela['preco'] = tabela['preco'].apply(lambda x: str(x).replace(".",'').replace("R$",''))
tabela['preco'] = pd.to_numeric(tabela['preco'].apply(lambda x: str(x).replace(",",'.')))
tabela['desconto'] = pd.to_numeric(tabela['desconto'].apply(lambda x: str(x).replace("%",'')))

tabela = tabela.sort_values(by=['desconto'],ascending=False)

tabela.head(15)

,marca,preco,desconto,link
1244,Tênis Coca Cola Loud Elite Preto e Branco,159.9,50,https://www.futfanatics.com.br/tenis-coca-cola...
1057,Tênis Fila Attrek Marinho e Vermelho,199.9,47,https://www.futfanatics.com.br/tenis-fila-attr...
1245,Tênis Coca Cola X-Fly Cage Grafite,149.9,46,https://www.futfanatics.com.br/tenis-coca-cola...
1038,Tênis Puma Wired Run Speckles BDP Marinho,219.9,45,https://www.futfanatics.com.br/tenis-puma-wire...
905,Tênis Reserva Ultra Basic Azul,249.9,44,https://www.futfanatics.com.br/tenis-reserva-u...
234,Tênis Under Armour HOVR Infinite 2 Coral e Preto,499.9,44,https://www.futfanatics.com.br/tenis-under-arm...
1296,Tênis Olympikus Orion Preto e Vermelho,129.9,43,https://www.futfanatics.com.br/tenis-olympikus...
1064,Tênis Redley Originals Explore Cano Médio Bege,199.9,42,https://www.futfanatics.com.br/tenis-redley-or...
998,Tênis Puma Wired Run BDP Azul Marinho,219.9,42,https://www.futfanatics.com.br/tenis-puma-wire...
1036,Tênis Puma Wired Run Speckles BDP Cinza,219.9,42,https://www.futfanatics.com.br/tenis-puma-wire...


In [6]:
tabela.to_csv('precoTenis.csv', encoding='utf-8', sep=';', index=False)